In [23]:
import open3d as o3d
import numpy as np
import sys
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib


In [24]:
sys.path.append('../sample')
from readdatas import *

In [25]:
path = 'D:\project\Point_Datas\Point Cloud Data\Corner.ply'
pcd =  ReadDatas(path)

In [26]:
gpf = GpfGroundExtractor()
gpf.set_data(pcd)
gpf._process_data()

In [35]:
class SVM_FPFH(BaseAlgorithm):

    def __init__(self, data, path):
        self.data = data;
        self.ground = data.ground
        self.ground_o3d = self.np_to_o3d(self.ground)
        self.ground_nor = np.asarray(self.ground_o3d.normals)
        self.feature_vector = self.get_feature_vector()
        self.path = path

    def _process_data(self):
        clf = joblib.load(self.path)
        print('hellow')
        y_pred_all = clf.predict(self.feature_vector)
        index = np.array(range(len(pcd.ground)))
        index = index[y_pred_all]
        self.no_side = self.ground_o3d.select_by_index(index)
        self.data.paves = self.euclidean_cluster(self.no_side)   # 欧拉点数目大于150个的点都是avaible for pave. 大约6000个点


    def euclidean_cluster(self,cloud, tolerance=0.2):
        seed = self.seed_select(cloud)
        no_paves = self.find_no_paves(cloud)
        paves = np.asarray(cloud.points)
        no_paves = self.find_index(paves, no_paves)
        kdtree = o3d.geometry.KDTreeFlann(cloud)
        paves = np.array(seed)
        while len(seed) > 0 :
            seed_now = seed[0]
            k, idx, _ = kdtree.search_radius_vector_3d(cloud.points[seed_now], tolerance)
            if k == 1 :
                continue
            idx = np.array(idx)
            idx = idx[np.isin(idx, paves, invert=True)]
            idx = idx[np.isin(idx, no_paves, invert=True)]
            paves = np.append(paves, idx)
            seed = np.append(seed,idx)
            seed = np.delete(seed,[0])
        return paves

        

    def get_feature_vector(self):
        # 计算法向量
        self.ground_o3d.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

        # 计算FPFH描述符
        radius_feature = 0.25
        fpfh = o3d.pipelines.registration.compute_fpfh_feature(
            self.ground_o3d,
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
        feature_vector = fpfh.data.T
        return feature_vector

In [36]:
path = 'SVM_model_fpfh.pkl'

In [37]:
svm = SVM_FPFH(pcd,path)

In [38]:
feature_vector = svm.feature_vector

In [40]:
svm._process_data()

hellow


In [ ]:

reg = ReGrowSegment(pcd)
reg._process_data()

In [ ]:
paves = ground_o3d.select_by_index(pcd.paves)
o3d.visualization.draw_geometries([paves])

In [ ]:
y = np.full(len(pcd.ground),False,dtype=bool)
y[pcd.paves] = True

In [ ]:
# 创建一个SVM模型，使用RBF核函数
clf = svm.SVC(kernel='rbf', C=1, gamma=0.5)

# 训练模型
clf.fit(feature_vector, y)

In [32]:

import joblib
# joblib.dump(clf, 'SVM_model_fpfh.pkl')
clf = joblib.load('SVM_model_fpfh.pkl')

In [ ]:
# y_pred = clf.predict(feature_vector[pcd.paves])
y_pred_all = clf.predict(feature_vector)

In [ ]:
accuracy_all = accuracy_score(y, y_pred_all)

# 输出准确率
print("Accuracy: %0.2f" % accuracy_all)

In [ ]:
index = np.array(range(len(pcd.ground)))
index = index[y_pred_all]

In [ ]:
paves = ground_o3d.select_by_index(index)
o3d.visualization.draw_geometries([paves])